In [ ]:
import pickle
import warnings
import json

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import uproot
from itables import init_notebook_mode
from matplotlib.colors import LogNorm
from matplotlib.ticker import AutoMinorLocator, MaxNLocator
from scipy import stats
from skimage.measure import LineModelND, ransac
from sklearn.cluster import DBSCAN
from sklearn.linear_model import RANSACRegressor
from skspatial.objects import Cylinder, Line, Triangle, Point
from tqdm.auto import tqdm

init_notebook_mode(all_interactive=True)

In [ ]:
light_file = "../Data/rwf_0cd913fa_20230706_191437.data_array.root"
charge_file = "../Data/evd_self_trigger-packets-2023_07_06_19_14_CEST_validated.root"
sipm_map_file = "sipm_sn_ch_to_xy.json"
detector_z = 300
detector_x = 128
detector_y = 160

In [ ]:
# Uproot
def load_charge(file_name, events=None):
    with uproot.open(file_name) as f:
        charge_df = f["events"].arrays(library="pd").set_index("eventID")
        if events is not None:
            charge_df = charge_df.loc[events]

    return charge_df


def load_light(file_name, deco=True, events=None, mask=True):
    light_df = pd.DataFrame()
    with uproot.open(file_name) as f:
        if deco:
            tree = f["decowave"]
        else:
            tree = f["rwf_array"]

        for idx, arrays in enumerate(tree.iterate(library="np")):
            df = pd.DataFrame.from_dict(arrays, orient="index").T
            df.dropna()
            if events is not None:
                df = df[df["event"].isin(events)]

            if mask:
                df = df[
                    df[["sn", "ch"]].apply(lambda x: get_sipm_mask(x[0], x[1]), axis=1)
                ]

            if df.empty:
                continue

            df[["x", "y"]] = df[["sn", "ch"]].apply(
                lambda x: pd.Series(sipm_to_xy(x[0], x[1])), axis=1
            )

            if deco:
                df["rwf"] = df["decwfm"]

            # df[["integral", "properties"]] = df["rwf"].apply(
            #     lambda x: pd.Series(
            #         integrate_peaks(x) if all(np.isnan(x)) > 0 else (np.nan, {})
            #     )
            # )
            # df["peak"] = df["properties"].apply(
            #     lambda x: max(x["peak_heights"])
            #     if "peak_heights" in x and len(x["peak_heights"]) > 0
            #     else np.nan
            # )

            df = df[
                [
                    "event",
                    "sn",
                    "ch",
                    # "peak",
                    # "integral",
                    "rwf"
                    # "x",
                    # "y",
                    # "tai_ns",
                    # "tslpps_ns",
                    # "ntp_ms",
                ]
            ]
            light_df = pd.concat([light_df, df], ignore_index=True)

    return light_df

In [ ]:
# SiPMs mapping
sipm_map = None


def sipm_to_xy(sn, ch):
    global sipm_map
    if sipm_map is None:
        with open(sipm_map_file, "r") as f:
            sipm_map = json.load(f)

    xy = sipm_map.get(str(sn), {}).get(str(ch), None)
    if xy is None:
        return None
    else:
        x = xy[0] + 64
        y = xy[1] - 16
        return (x, y)


# Check if SiPMs on anode area
def get_sipm_mask(sn, ch):
    xy = sipm_to_xy(sn, ch)
    # return True
    if xy is None:
        return False
    else:
        return (
            xy[0] > -detector_x / 2
            and xy[0] < detector_x / 2
            and xy[1] < detector_y / 2
            and xy[1] > -detector_y / 2
        )

In [ ]:
charge_df = load_charge(charge_file)
# charge_df = pd.read_csv("charge_df.bz2")

In [ ]:
light_df = load_light(light_file, events=None, deco="deco" in light_file)
# light_df = pd.read_csv("light_df.bz2")

In [ ]:
import numpy as np
import scipy.signal as signal

# Peak finding algorithm for integration
# def integrate_peaks(waveform, buffer_size=10, height=0.1, prominence=0.05):
#     # Find peaks in the filtered waveform
#     peaks, properties = signal.find_peaks(
#         waveform, height=height, prominence=prominence
#     )

#     # Create an index range for the peak region with the buffer
#     integration_result = 0

#     for peak in peaks:
#         start_index = max(0, peak - buffer_size)
#         end_index = min(len(waveform), peak + buffer_size)

#         # Extract the peak region with the buffer
#         peak_region = waveform[start_index:end_index]

#         # Integrate the peak region
#         integration_result += np.trapz(peak_region)

#     return integration_result, properties


def integrate_peaks(waveform, buffer_size=10, height=0.1, prominence=0.05):
    # Find peaks in the filtered waveform
    peaks, properties = signal.find_peaks(
        waveform, height=height, prominence=prominence
    )

    integration_result = 0
    start_index = 0  # Initialize the start index
    end_index = 0  # Initialize the end index

    for peak in peaks:
        # Determine the potential start and end indices
        potential_start = max(0, peak - buffer_size)
        potential_end = min(len(waveform), peak + buffer_size)

        # If the potential start is within the current peak region, update the end index
        if potential_start <= end_index:
            end_index = potential_end
        else:
            # Integrate the previous peak region and update indices for the new peak
            peak_region = waveform[start_index:end_index]
            integration_result += np.trapz(peak_region)
            start_index = potential_start
            end_index = potential_end

    # Integrate the last peak region
    peak_region = waveform[start_index:end_index]
    integration_result += np.trapz(peak_region)

    # if integration_result < 0:
    #     # print(idx)
    #     plt.plot(waveform)
    #     print(integration_result)
    #     # plt.legend()
    #     plt.show()

    return integration_result, properties

In [ ]:
integrals = []
peaks = []
for idx, waveform in enumerate(tqdm(light_df["rwf"])):
    integral, properties = integrate_peaks(waveform)
    peak = max(properties["peak_heights"]) if len(properties["peak_heights"]) > 0 else 0
    integrals.append(integral)
    peaks.append(peaks)
    # print(integral,peak)

integrals = np.array(integrals)
peaks = np.array(peaks)

In [ ]:
for rwf in light_df[(light_df["event"]==49703) & (light_df["ch"]==11)]["rwf"]:
    plt.plot(rwf)
    plt.show()